In [1]:
import numpy as np
import pandas as pd

In [2]:
data_preprocessed = pd.read_csv("Absenteeism_preprocessed.csv")

In [3]:
data_preprocessed["Absenteeism Time in Hours"].median()

3.0

In [4]:
targets = np.where(data_preprocessed["Absenteeism Time in Hours"] > 
                   data_preprocessed["Absenteeism Time in Hours"].median(), 1, 0)

In [5]:
data_preprocessed["Excessive Absenteeism"] = targets

In [6]:
data_with_targets = data_preprocessed.drop(["Absenteeism Time in Hours", 'Day of the Week',
                                           'Daily Work Load Average', 'Distance to Work'], axis=1)

In [7]:
unscaled_inputs = data_with_targets.iloc[:, :-1]

In [8]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler()
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [9]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [10]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [11]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [12]:
absenteeism_scaler.fit(unscaled_inputs)

CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [13]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, test_size=0.2, 
                                                    random_state=111)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [18]:
reg = LogisticRegression()
reg.fit(x_train, y_train)

LogisticRegression()

In [19]:
reg.score(x_train, y_train)

0.7803571428571429

In [20]:
model_outputs = reg.predict(x_train)

In [21]:
reg.score(x_test, y_test)

0.7571428571428571

In [22]:
import pickle 

In [23]:
with open("model", "wb") as file:
    pickle.dump(reg, file)

In [24]:
with open('scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)